# RegNet-v4

In [ ]:
exp_type = 'null' #null/fixed/dynamic_greedy/dynamic_kmeans
dataset_class_toremove = 'all' #all/[class_name] e.g. dolphin
dataset_portion = 0.05

experiment_name = 'v4_' + exp_type + '_' + dataset_class_toremove + str(dataset_portion)
print(experiment_name)

In [ ]:
from commons import *
from net import Net
from trainer import Trainer
import evaluate
from data_loader import DataClass
from data_collector import DataCollector
from utils import *

%matplotlib notebook
#If you need to change the above line, change matlibplotlib.use() in plotter.py accordingly

In [ ]:
pin_mem_perm = False #TODO: should be True
num_training_epochs = 150

learning_rate, batch_size = 0.01, 128
update_parents_enc, dynamic_hierarchy_flag = True, True
hierarchy_update_method = None
parent_enc_update_interval, hierarchy_update_interval = 50, 1000
kmeans_num_clusters = 25
do_print, do_plot = True, False

myassertList(exp_type, ['null', 'fixed', 'dynamic_greedy', 'dynamic_kmeans'])
if exp_type == 'null':
    update_parents_enc, dynamic_hierarchy_flag = False, False
elif exp_type == 'fixed':
    update_parents_enc, dynamic_hierarchy_flag = True, False
elif exp_type == 'dynamic_greedy':
    update_parents_enc, dynamic_hierarchy_flag = True, True
    hierarchy_update_method = 'greedy'
elif exp_type == 'dynamic_kmeans':
    update_parents_enc, dynamic_hierarchy_flag = True, True
    hierarchy_update_method = 'kmeans'
    kmeans_num_clusters = 25

data_class = DataClass('data/')
data_collector = DataCollector('tmp/', experiment_name)

## Define the model

In [ ]:
model = Net()
model = model.cuda() if use_cuda else model
model

## Instantiate the trainer class

In [ ]:
trainer = Trainer(data_collector,
                  learning_rate = learning_rate,
                  use_hierarchy = update_parents_enc,
                  dynamic_hierarchy_flag = dynamic_hierarchy_flag,
                  parent_enc_update_interval = parent_enc_update_interval,
                  hierarchy_update_interval = hierarchy_update_interval,
                  hierarchy_update_method = hierarchy_update_method,
                  kmeans_num_clusters = kmeans_num_clusters)

## Hyper-parameter tuning

In [ ]:
# To actually perform hyper-parameter tuning, uncomment the following lines.

#data_class.load(batch_size, validation_portion=0.3,
#                portion_to_keep = data_class.updatePortionToKeep(dataset_class_toremove, dataset_portion),
#                pin_mem_perm=pin_mem_perm)
#(lambda_1_best, lambda_2_best) = trainer.hyperParamTuning(model, data_class,
#                                                          [2**i for i in range(-15,-5)],
#                                                          [2**i for i in range(-15,-5)])
lambda_1_best = 2**(-12)
lambda_2_best = 2**(-8)
print('Hyper-parameter tuning finished! \t lambda_1 = %f, lambda_2 = %f' %(lambda_1_best, lambda_2_best))

## Train the network
Using the best hyper-parameter found

In [ ]:
data_class.load(batch_size, validation_portion=0.0,
                portion_to_keep = data_class.updatePortionToKeep(dataset_class_toremove, dataset_portion),
                pin_mem_perm=pin_mem_perm)
trainer.train(model, data_class,
              lambda_1=lambda_1_best, lambda_2=lambda_2_best,
              num_epochs=num_training_epochs,
              do_print=False, do_plot=False)

## Store the final model and related data on disk

In [ ]:
data_collector.finalizeModel(state='final', dst_root='results/')
data_collector.finilizeHierarchy(hierarchy=trainer.getNamedHierarchy(data_class), dst_root='results/')
#data_collector.finalizeLosses(state='running', dst_root='results/')

## Plot training error acheived during training

In [ ]:
'''
from plotter import Plotter

plotter = Plotter(2,1)
[plot_losses, plot_seen_samples] = data_collector.loadLosses('running')
plot_avg_losses = []
for i in range(len(plot_losses)):
    plot_avg_losses.append(sum(plot_losses[:i+1])/(i+1))

plotter.update(plot_seen_samples, [plot_losses, plot_avg_losses], 0)
plotter.update(plot_seen_samples[-50:], [plot_losses[-50:]], 1)
'''

## Print the final learnt hierarchy

In [ ]:
#Get the hierarchy from trainer
named_hierarchy = trainer.getNamedHierarchy(data_class)
#Or load it from the stored file
#named_hierarchy = np.load('results/'+experiment_name+'/models/final_hierarchy.npy').take(0)
for parent, children in named_hierarchy.items():
    print('Superclass {0}'.format(parent+1), ':')
    print(children)

## Evaluate the network on test set

In [ ]:
accuracy = evaluate.top1(model, data_class.testloader)
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * accuracy))

accuracy_topk = evaluate.topK(model, data_class.testloader, 5)
print('Accuracy of the network on the 10000 test images (top-5): %d %%' % (
    100 * accuracy_topk))

## Evaluate the network on training set

In [ ]:
accuracy = evaluate.top1(model, data_class.trainloader)
print('Accuracy of the network on training images: %d %%' % (
    100 * accuracy))

accuracy_topk = evaluate.topK(model, data_class.trainloader, 5)
print('Accuracy of the network on training images (top-5): %d %%' % (
    100 * accuracy_topk))